In [5]:
import pickle, numpy as np, keras, time
import attention, attention_with_context
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import models

Using TensorFlow backend.
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [616]:
with open('/data/jrgillick/x_train.pkl') as f:
    X_train = pickle.load(f)

In [617]:
with open('/data/jrgillick/x_test.pkl') as f:
    X_test = pickle.load(f)

In [618]:
with open('/data/jrgillick/y_train.pkl') as f:
    y_train = pickle.load(f)

In [619]:
with open('/data/jrgillick/y_test.pkl') as f:
    y_test = pickle.load(f)

In [2]:
t0 = time.time()
with open('/data/jrgillick/individual_feature_lists_skip_thoughts.pkl') as f:
    individual_feature_lists = pickle.load(f)
print "load in " + str(time.time() - t0)

load in 38.2689790726


In [6]:
with open('/data/jrgillick/individual_feature_lists_no_n_grams.pkl','rb') as f:
    individual_feature_lists = pickle.load(f)

In [10]:
individual_feature_lists[0].keys()

['euphony',
 'labels',
 'word_overlap',
 'phone',
 'substring',
 'liu',
 'cosine',
 'rst',
 'audio',
 'id']

In [16]:
#with open('/data/jrgillick/individual_feature_lists_no_n_grams.pkl','wb') as f:
#    pickle.dump(individual_feature_lists, f)

In [3]:
len(individual_feature_lists)

NameError: name 'individual_feature_lists' is not defined

In [19]:
from sklearn.model_selection import KFold

In [12]:
speaker_names = ['donald_trump',
 'hilary_clinton',
 'bernie_sanders',
 'ted_cruz',
 'marco_rubio',
 'john_kasich',
 'barack_obama',
 'bill_clinton',
 'joe_biden',
 'mike_pence',
 'carly_fiorina',
 'jeb_bush',
 'rand_paul',
 'gary_johnson',
 'chris_christie',
 'rick_santorum']

In [14]:
def normalize_data(X_train, X_test):
    new_X_train = np.copy(X_train)
    new_X_test = np.copy(X_test)
    for i in xrange(new_X_train.shape[1]):
        if set(new_X_train[:,i]) != set([0,1]) and set(new_X_train[:,i]) != set([0]) and set(new_X_train[:,i]) != set([1]):
            #print 'norming'
            #print set(new_X_train[:,i])
            mean = np.mean(new_X_train[:,i])
            new_X_train[:,i] -= mean
            new_X_test[:,i] -=mean
            std = np.std(X_train[:,i])
            if std > 1e-2:
                new_X_train[:,i] /= std
                new_X_test[:,i] /= std
    return new_X_train, new_X_test

def get_train_and_test_sets(speaker_name):
    train_set = []
    test_set = []
    for i in range(len(individual_feature_lists)):
        if speaker_name in individual_feature_lists[i]['id']:
            test_set.append(individual_feature_lists[i])
        else:
            train_set.append(individual_feature_lists[i])
    return train_set, test_set

def get_features(data_set, feature_names):
    all_feats = []
    for i in range(len(data_set)):
        feats = [data_set[i][name] for name in feature_names]
        all_feats.append(np.hstack(feats))
    return all_feats

def get_labels(data_set):
    all_labels = []
    for i in range(len(data_set)):
        all_labels.append(data_set[i]['labels'])
    return all_labels

In [15]:
from models import *
def initialize_lstm_attention_model(input_dim, layer_sizes, dropout = 0.5, loss = 'binary_crossentropy', metrics = ['accuracy'], batch_norm = True, bidirectional = False):
    model = Sequential()
    model.add(Dropout(0.1,input_shape=(None, input_dim)))
    for i in range(len(layer_sizes)):
        return_sequences = False if i == len(layer_sizes) - 1 else True
        if bidirectional:
            model.add(Bidirectional(LSTM(layer_sizes[i], return_sequences=return_sequences, dropout=dropout),input_shape=(None, input_dim)))
        else:
            model.add(LSTM(layer_sizes[i],input_shape=(None,input_dim),return_sequences=True,dropout=dropout))
        if batch_norm: model.add(keras.layers.BatchNormalization())
        model.add(Activation("relu"))

    #model.add(Dense(10))
    #model.add(Activation("relu"))
    model.add(attention_with_context.AttentionWithContext())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer,loss=loss,metrics=metrics)
    return model

In [21]:
def run_neural_single_speaker(feature_set, phrase_count, verbose=True):
    all_true = []; all_pred = []
    C = None
    for speaker in speaker_names:
        speaker_true = []; speaker_pred = []
        print speaker + ": "
        other, speaker_data = get_train_and_test_sets(speaker)
        speaker_data = np.array(speaker_data)
        #speaker_labels = get_labels(speaker_data)
        
        if len(speaker_data) == 1:
            continue
        
        if len(speaker_data) > 10:
            kf = KFold(n_splits=10)
        else:
            kf = KFold(n_splits=len(speaker_data))
            
        for train_index, test_index in kf.split(speaker_data):
            train_set = speaker_data[train_index]
            test_set = speaker_data[test_index]
            #train_set, test_set = train_test_split(speaker_data, test_size=0.2, random_state=43)

            labels_train = get_labels(train_set)
            data_train = get_features(train_set,feature_set)

            labels_test = get_labels(test_set)
            data_test = get_features(test_set,feature_set)

            current_train_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input(data_train,labels_train,phrase_count=phrase_count)
            current_test_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input(data_test,labels_test,phrase_count=phrase_count)

            X_train = np.array(current_train_formatted_feats).astype('float64')
            y_train = np.array(current_train_formatted_labs)

            X_test = np.array(current_test_formatted_feats).astype('float64')
            y_test = np.array(current_test_formatted_labs)

            X_train, X_test = normalize_data(X_train,X_test)
            #print X_train.shape
            #print X_test.shape
            
            if len(y_train) > 0 and len(y_test) > 0:
                lstm_model = initialize_lstm_attention_model(X_train.shape[1]/phrase_count,[25],bidirectional=False,dropout=0.6,batch_norm=True) 
                lstm_model.fit(X_train.reshape(X_train.shape[0],phrase_count,-1),np.array(y_train),batch_size=512,epochs=15,shuffle=True,verbose=False)

                y_true, y_pred = models.evaluate_model(lstm_model, X_test.reshape(X_test.shape[0],phrase_count,-1), y_test, model_type='keras')

                all_true += list(y_true)
                all_pred += list(y_pred)
                

            print
    return all_true, all_pred

In [28]:
def run_neural_cross_validation(feature_set, phrase_count):
    all_true = []; all_pred = []
    num_iters = None
    for speaker in speaker_names:
        print speaker + ": "
        train_set, test_set = get_train_and_test_sets(speaker)

        labels_train = get_labels(train_set)
        data_train = get_features(train_set,feature_set)

        labels_test = get_labels(test_set)
        data_test = get_features(test_set,feature_set)

        current_train_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input(data_train,labels_train,phrase_count=phrase_count)
        current_test_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input(data_test,labels_test,phrase_count=phrase_count)

        X_train = np.array(current_train_formatted_feats).astype('float64')
        y_train = np.array(current_train_formatted_labs)
        
        print X_train.shape

        X_test = np.array(current_test_formatted_feats).astype('float64')
        y_test = np.array(current_test_formatted_labs)

        X_train, X_test = normalize_data(X_train,X_test)
        
        lstm_model = initialize_lstm_attention_model(X_train.shape[1]/phrase_count,[100],bidirectional=False,dropout=0.5,batch_norm=True) 
        lstm_model.fit(X_train.reshape(X_train.shape[0],phrase_count,-1),np.array(y_train),batch_size=1024,epochs=25,shuffle=True,verbose=False)
        
        #if num_iters is None:
        #    
        #    model = models.train_cv_logistic_regression_faster(X_train,y_train)
        #    C = model.best_params_['C']
        
        #lstm_model.fit(X_train.reshape(X_train.shape[0],5,-1),np.array(y_train),batch_size=512,epochs=15,shuffle=True)
        y_true, y_pred = models.evaluate_model(lstm_model, X_test.reshape(X_test.shape[0],phrase_count,-1), y_test, model_type='keras')
        
        all_true += list(y_true)
        all_pred += list(y_pred)
        print
    return all_true, all_pred

In [26]:
#all_true, all_pred = run_neural_single_speaker(['audio','liu','euphony','phone','cosine','substring','word_overlap','rst'],phrase_count=3)

In [27]:
print_results(total_true, total_pred)

Accuracy: 0.625 +/- 0.005 (21930/35092) | Precision: 0.628 | Recall: 0.613 | F1: 0.621


In [29]:
total_true, total_pred = run_neural_cross_validation(['audio','liu','euphony'],phrase_count=3)

donald_trump: 
(22376, 270)
Accuracy: 0.597 +/- 0.009 (7588/12716) | Precision: 0.621 | Recall: 0.497 | F1: 0.552

hilary_clinton: 
(27942, 270)
Accuracy: 0.650 +/- 0.011 (4647/7150) | Precision: 0.627 | Recall: 0.741 | F1: 0.679

bernie_sanders: 
(28630, 270)
Accuracy: 0.595 +/- 0.012 (3844/6462) | Precision: 0.635 | Recall: 0.445 | F1: 0.524

ted_cruz: 
(33040, 270)
Accuracy: 0.600 +/- 0.021 (1231/2052) | Precision: 0.685 | Recall: 0.369 | F1: 0.480

marco_rubio: 
(33530, 270)
Accuracy: 0.649 +/- 0.024 (1013/1562) | Precision: 0.675 | Recall: 0.572 | F1: 0.620

john_kasich: 
(34476, 270)
Accuracy: 0.661 +/- 0.037 (407/616) | Precision: 0.677 | Recall: 0.614 | F1: 0.644

barack_obama: 
(33266, 270)
Accuracy: 0.629 +/- 0.022 (1149/1826) | Precision: 0.685 | Recall: 0.479 | F1: 0.564

bill_clinton: 
(34518, 270)
Accuracy: 0.666 +/- 0.039 (382/574) | Precision: 0.675 | Recall: 0.638 | F1: 0.656

joe_biden: 
(34564, 270)
Accuracy: 0.680 +/- 0.040 (359/528) | Precision: 0.686 | Recall: 0.6

In [25]:
def print_results(total_true, total_pred):
    total_correct = np.sum(np.array(total_true) == np.array(total_pred))
    acc=float(total_correct) / len(total_true)
    std=math.sqrt( (acc * (1-acc)) / len(total_true) )
    precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(total_true, total_pred)]
    #print "Accuracy: %.3f +/- %.3f (%s/%s)" % (acc, 1.96*std, total_correct, len(total_true))
    print "Accuracy: %.3f +/- %.3f (%s/%s) | Precision: %.3f | Recall: %.3f | F1: %.3f" % (acc, 1.96*std, total_correct, len(total_true), precision, recall, f1)


In [20]:
all_true, all_pred = run_neural_cross_validation(['audio','liu','euphony','phone'],phrase_count=1)

donald_trump: 
(22678, 186)
12608/12846 [============================>.] - ETA: 0sAccuracy: 0.599 +/- 0.008 (7696/12846) | Precision: 0.600 | Recall: 0.594 | F1: 0.597

hilary_clinton: 
(28264, 186)
6880/7260 [===========================>..] - ETA: 0sAccuracy: 0.649 +/- 0.011 (4713/7260) | Precision: 0.624 | Recall: 0.751 | F1: 0.682

bernie_sanders: 
(29006, 186)
6368/6518 [============================>.] - ETA: 0sAccuracy: 0.627 +/- 0.012 (4085/6518) | Precision: 0.629 | Recall: 0.619 | F1: 0.624

ted_cruz: 
(33442, 186)
2080/2082 [============================>.] - ETA: 0sAccuracy: 0.599 +/- 0.021 (1247/2082) | Precision: 0.635 | Recall: 0.466 | F1: 0.537

marco_rubio: 
(33938, 186)
1248/1586 [======================>.......] - ETA: 0sAccuracy: 0.651 +/- 0.023 (1033/1586) | Precision: 0.661 | Recall: 0.622 | F1: 0.641

john_kasich: 
(34886, 186)
416/638 [==================>...........] - ETA: 1s Accuracy: 0.676 +/- 0.036 (431/638) | Precision: 0.676 | Recall: 0.674 | F1: 0.675

barack

In [22]:
total_correct = np.sum(np.array(all_true) == np.array(all_pred))
acc=float(total_correct) / len(all_true)
std=math.sqrt( (acc * (1-acc)) / len(all_true) )
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(all_true, all_pred)]
print "Accuracy: %.3f +/- %.3f (%s/%s) | Precision: %.3f | Recall: %.3f | F1: %.3f" % (acc, 1.96*std, total_correct, len(all_true), precision, recall, f1)

Accuracy: 0.623 +/- 0.005 (22142/35524) | Precision: 0.621 | Recall: 0.632 | F1: 0.627


In [396]:
with open('/data/jrgillick/x_train_deltas.pkl','rb') as f:
    X_train_deltas = pickle.load(f)
    
with open('/data/jrgillick/x_test_deltas.pkl','rb') as f:
    X_test_deltas = pickle.load(f)

with open('/data/jrgillick/y_train_deltas.pkl','rb') as f:
    y_train_deltas = pickle.load(f)
    
with open('/data/jrgillick/y_test_deltas.pkl','rb') as f:
    y_test_deltas = pickle.load(f)

In [604]:
with open('/data/jrgillick/x_train_rst.pkl','rb') as f:
    X_train = pickle.load(f)
    
with open('/data/jrgillick/x_test_rst.pkl','rb') as f:
    X_test = pickle.load(f)

with open('/data/jrgillick/y_train_rst.pkl','rb') as f:
    y_train = pickle.load(f)
    
with open('/data/jrgillick/y_test_rst.pkl','rb') as f:
    y_test = pickle.load(f)

In [ ]:
X_train.shape

In [725]:
def normalize_data(X_train, X_test):
    new_X_train = np.copy(X_train)
    new_X_test = np.copy(X_test)
    for i in xrange(new_X_train.shape[1]):
        if set(new_X_train[:,i]) != set([0,1]):
            mean = np.mean(new_X_train[:,i])
            new_X_train[:,i] -= mean
            new_X_test[:,i] -=mean
            std = np.std(X_train[:,i])
            new_X_train[:,i] /= std
            new_X_test[:,i] /= std
    return new_X_train, new_X_test

In [717]:
t_X = np.random.uniform(0,1,(3,4))
t_X

array([[ 0.16932148,  0.36295728,  0.75744165,  0.62218308],
       [ 0.01858241,  0.3735383 ,  0.36718772,  0.1485428 ],
       [ 0.0574262 ,  0.64471333,  0.6655188 ,  0.2945661 ]])

In [719]:
t_X[0,1] = t_X[1,1] = 0
t_X[2,1] = 1
t_X

array([[ 0.16932148,  0.        ,  0.        ,  0.62218308],
       [ 0.01858241,  0.        ,  0.36718772,  0.1485428 ],
       [ 0.0574262 ,  1.        ,  0.6655188 ,  0.2945661 ]])

In [723]:
normalize_data(t_X,t_X)[0]

array([[ 1.36997627,  0.        , -1.26472911,  1.34862213],
       [-0.98891867,  0.        ,  0.08432696, -1.0429759 ],
       [-0.3810576 ,  1.        ,  1.18040215, -0.30564623]])

In [724]:
t_X

array([[ 0.16932148,  0.        ,  0.        ,  0.62218308],
       [ 0.01858241,  0.        ,  0.36718772,  0.1485428 ],
       [ 0.0574262 ,  1.        ,  0.6655188 ,  0.2945661 ]])

In [702]:
X_train.shape

(28132, 1550)

In [712]:
set(X_train[:,-17]) == set([0,1])

False

In [622]:
ff_model = models.initialize_feed_forward_model(input_dim=1550,layer_sizes=[50],dropout=0.7)

In [667]:
ff_model.fit(X_train,np.array(y_train),batch_size=512,epochs=3)

Epoch 1/3
28132/28132 [==============================] - 0s - loss: 0.5613 - acc: 0.7108     
Epoch 2/3
28132/28132 [==============================] - 0s - loss: 0.5644 - acc: 0.7065     
Epoch 3/3
28132/28132 [==============================] - 0s - loss: 0.5632 - acc: 0.7090     


In [668]:
ff_model.evaluate(X_test,np.array(y_test),batch_size=128)

5760/6514 [=========================>....] - ETA: 0s

[0.65558727452972387, 0.61728584543122156]

In [350]:
models.evaluate_model(ff_model, X_test, y_test,model_type='keras')

6496/6606 [============================>.] - ETA: 0sAccuracy: 0.639 +/- 0.012 (4221/6606) | Precision: 0.635 | Recall: 0.652 | F1: 0.644


(0.6389645776566758,
 0.63531839622641506,
 0.65243717832273695,
 0.64376400298730396)

In [260]:
X_train.shape

(28486, 930)

In [261]:
930/3.

310.0

In [605]:
model = models.train_cv_logistic_regression(X_train,y_train)

In [606]:
# Just RST at t0
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.546 +/- 0.012 (3654/6692) | Precision: 0.562 | Recall: 0.420 | F1: 0.481


(0.5460251046025104,
 0.561550759392486,
 0.41990436341900778,
 0.48050615595075236)

In [589]:
def get_continuous_rst_resolution_val(X):
    last_cols = X[:,-10:]
    output_vals = []
    for i in range(len(last_cols)):
        if len(last_cols[i].nonzero()[0]) == 0:
            output_vals.append(0)
        else:
            output_vals.append(last_cols[i].nonzero()[0].argmax())
    return np.array(output_vals).reshape((-1,1))

def convert_rst_to_continuous(X):
    first_cols = X[:,0:-10]
    last_cols = get_continuous_rst_resolution_val(X)
    return np.hstack([first_cols,last_cols])

In [592]:
#X_train = convert_rst_to_continuous(X_train)
#X_test = convert_rst_to_continuous(X_test)

In [596]:
X_test.shape

(6692, 50)

In [586]:
np.array(continuous_vals).reshape((-1,1)).shape

(28832, 1)

In [387]:
# all feats with deltas, 3 phrases
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.631 +/- 0.012 (4169/6606) | Precision: 0.632 | Recall: 0.629 | F1: 0.630


(0.6310929458068423,
 0.63177115033475351,
 0.62851952770208896,
 0.63014114433146151)

In [410]:
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.631 +/- 0.012 (4169/6606) | Precision: 0.632 | Recall: 0.629 | F1: 0.630


(0.6310929458068423,
 0.63177115033475351,
 0.62851952770208896,
 0.63014114433146151)

In [534]:
with open('all_feature_names.pkl','rb') as f:
    all_feature_names = pickle.load(f)

In [607]:
all_feature_names = ['rst_cat_N-antithesis',
 'rst_cat_N-attribution',
 'rst_cat_N-circumstance',
 'rst_cat_N-comment',
 'rst_cat_N-comparison',
 'rst_cat_N-concession',
 'rst_cat_N-condition',
 'rst_cat_N-consequence',
 'rst_cat_N-contrast',
 'rst_cat_N-definition',
 'rst_cat_N-disjunction',
 'rst_cat_N-elaboration',
 'rst_cat_N-evidence',
 'rst_cat_N-example',
 'rst_cat_N-explanation',
 'rst_cat_N-hypothetical',
 'rst_cat_N-list',
 'rst_cat_N-manner',
 'rst_cat_N-means',
 'rst_cat_N-purpose',
 'rst_cat_N-question',
 'rst_cat_N-reason',
 'rst_cat_N-restatement',
 'rst_cat_N-result',
 'rst_cat_N-same_unit',
 'rst_cat_N-sequence',
 'rst_cat_N-temporal',
 'rst_cat_N-textualorganization',
 'rst_cat_N-topic',
 'rst_cat_S-antithesis',
 'rst_cat_S-attribution',
 'rst_cat_S-circumstance',
 'rst_cat_S-comment',
 'rst_cat_S-comparison',
 'rst_cat_S-concession',
 'rst_cat_S-condition',
 'rst_cat_S-consequence',
 'rst_cat_S-definition',
 'rst_cat_S-elaboration',
 'rst_cat_S-example',
 'rst_cat_S-explanation',
 'rst_cat_S-hypothetical',
 'rst_cat_S-manner',
 'rst_cat_S-means',
 'rst_cat_S-purpose',
 'rst_cat_S-reason',
 'rst_cat_S-restatement',
 'rst_cat_S-result',
 'rst_cat_S-temporal',
 'rst_final_count>0',
 'rst_final_count>1',
 'rst_final_count>2',
 'rst_final_count>3',
 'rst_final_count>4',
 'rst_final_count>5',
 'rst_final_count>6',
 'rst_final_count>7',
 'rst_final_count>8',
 'rst_final_count>9']

In [608]:
len(all_feature_names)

59

In [609]:
coefs = model.best_estimator_.coef_[0]

In [610]:
len(coefs)

59

In [611]:
zip(np.array(all_feature_names)[np.argsort(coefs)],coefs[np.argsort(coefs)])

[('rst_cat_S-reason', -0.080807832271602112),
 ('rst_cat_N-reason', -0.073367700923741674),
 ('rst_cat_N-temporal', -0.060824950389316561),
 ('rst_cat_N-sequence', -0.032930750757426797),
 ('rst_cat_N-attribution', -0.029187086181217851),
 ('rst_final_count>5', -0.020056612614410618),
 ('rst_final_count>1', -0.019377843396127557),
 ('rst_cat_N-concession', -0.01894895793585763),
 ('rst_cat_S-antithesis', -0.012867169157696324),
 ('rst_cat_S-circumstance', -0.0091779903554919471),
 ('rst_final_count>6', -0.0086567385876791109),
 ('rst_cat_S-concession', -0.0068395373167089981),
 ('rst_cat_N-result', -0.0045257759789439434),
 ('rst_cat_S-result', -0.0045257759789439434),
 ('rst_cat_N-restatement', 0.0),
 ('rst_cat_S-definition', 0.0),
 ('rst_cat_S-restatement', 0.0),
 ('rst_cat_N-question', 0.0),
 ('rst_cat_N-hypothetical', 0.0),
 ('rst_cat_N-definition', 0.0),
 ('rst_cat_N-evidence', 0.0),
 ('rst_cat_S-hypothetical', 0.0),
 ('rst_final_count>7', 0.0019309184466439378),
 ('rst_cat_S-mean

In [394]:
model.best_params_

{'C': 0.001}

In [53]:
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.665 +/- 0.011 (4929/7416) | Precision: 0.680 | Recall: 0.622 | F1: 0.650


(0.6646440129449838,
 0.67982326951399119,
 0.62243797195253503,
 0.64986625369562157)

In [451]:
from models import *

In [452]:
def initialize_lstm_attention_model(input_dim, layer_sizes, dropout = 0.5, loss = 'binary_crossentropy', metrics = ['accuracy'], batch_norm = True, bidirectional = False):
    model = Sequential()
    model.add(Dropout(0.1,input_shape=(None, input_dim)))
    for i in range(len(layer_sizes)):
        return_sequences = False if i == len(layer_sizes) - 1 else True
        if bidirectional:
            model.add(Bidirectional(LSTM(layer_sizes[i], return_sequences=return_sequences, dropout=dropout),input_shape=(None, input_dim)))
        else:
            model.add(LSTM(layer_sizes[i],input_shape=(None,input_dim),return_sequences=True,dropout=dropout))
        if batch_norm: model.add(keras.layers.BatchNormalization())
        model.add(Activation("relu"))

    #model.add(Dense(10))
    #model.add(Activation("relu"))
    model.add(attention_with_context.AttentionWithContext())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer,loss=loss,metrics=metrics)
    return model

In [674]:
#lstm_model = models.initialize_lstm_model(310,[80],bidirectional=False,dropout=0.6,batch_norm=True)
lstm_model = initialize_lstm_attention_model(310,[80,40],bidirectional=False,dropout=0.6,batch_norm=True)

In [670]:
reload(attention)
reload(attention_with_context)

<module 'attention_with_context' from 'attention_with_context.pyc'>

In [52]:
X_train[:,0:164].shape

(94426, 164)

In [699]:
lstm_model.fit(X_train.reshape(X_train.shape[0],5,-1),np.array(y_train),batch_size=512,epochs=3,shuffle=True)

Epoch 1/3
28132/28132 [==============================] - 2s - loss: 0.6238 - acc: 0.6503     
Epoch 2/3
28132/28132 [==============================] - 2s - loss: 0.6185 - acc: 0.6586     
Epoch 3/3
28132/28132 [==============================] - 2s - loss: 0.6193 - acc: 0.6573     


In [700]:
lstm_model.evaluate(X_test.reshape(X_test.shape[0],5,-1),np.array(y_test),batch_size=1024)

6514/6514 [==============================] - 0s     


[0.63343355565631831, 0.64138778294746879]

In [497]:
models.evaluate_model(lstm_model,X_test.reshape(X_test.shape[0],3,-1),np.array(y_test),model_type='keras')

6496/6606 [============================>.] - ETA: 0sAccuracy: 0.633 +/- 0.012 (4181/6606) | Precision: 0.637 | Recall: 0.618 | F1: 0.627


(0.632909476233727,
 0.63693075483468498,
 0.61822585528307594,
 0.62743893071132273)

In [175]:
models.evaluate_model(model,scaler.transform(X_test),y_test,model_type='keras')

7456/7727 [===========================>..] - ETA: 0sAccuracy: 0.639 +/- 0.011 (4938/7727) | Precision: 0.638 | Recall: 0.637 | F1: 0.637


(0.6390578491005565,
 0.63787721123829344,
 0.63704858404780462,
 0.63746262836344736)

In [111]:
models.evaluate_model(lstm_model,lstm_x_test,lstm_y_test,model_type='keras')

7456/7633 [============================>.] - ETA: 0sAccuracy: 0.618 +/- 0.011 (4720/7633) | Precision: 0.641 | Recall: 0.555 | F1: 0.595


(0.6183676143063016,
 0.64088729016786572,
 0.55489229172073706,
 0.59479760745583521)